## Imports and useful funcs

In [1]:
runs_folder = 'Macro_TESA_Lin8'

In [2]:
from torch.utils.data import DataLoader
from Solvation_1.my_nets.Create_dataset import *
from Solvation_1.Vectorizers.vectorizers import *
from Solvation_1.my_nets.LinearNet import *
from Solvation_1.my_nets.net_func import *
import pickle

## Create dataset

In [3]:
table_v1 = pd.read_table(project_path('Solvation_1/Tables/SS_table_v3.tsv'))
dataset = SS_Dataset(table_v1, 'solvent_macro_props1', 'solute_TESA', normalize=(True, True, True))
len_data = dataset.__len__()
val_data = len_data // 10
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [len_data - val_data, val_data])

solvent_table = pd.read_table(project_path('Solvation_1/Tables/solvent_test_table_v3.tsv'))
solute_table = pd.read_table(project_path('Solvation_1/Tables/solute_test_table_v3.tsv'))
solvent_test_dataset = SS_Dataset(solvent_table, 'solvent_macro_props1', 'solute_TESA', normalize=(True, True, True))
solute_test_dataset = SS_Dataset(solute_table, 'solvent_macro_props1', 'solute_TESA', normalize=(True, True, True))

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

solvent_test_loader = DataLoader(solvent_test_dataset, batch_size=46, shuffle=False)
solute_test_loader = DataLoader(solute_test_dataset, batch_size=64, shuffle=False)
print(len(solute_test_loader.dataset))
print(len(solvent_test_loader.dataset))

length check-> Solvent: 2529, Solute: 2529, G_solv: 2529

Solvent
 std: tensor([ 0.0656,  0.2982,  0.2228, 24.6910, 28.1419,  0.3204,  0.2254]) 
 mean: tensor([ 1.4262,  0.1996,  0.2314, 48.5389, 17.9622,  0.1575,  0.0863])
Solute
 std: tensor([46.2081, 23.2195, 32.6769, 33.9146,  2.0706,  6.2217,  6.3596,  1.3146,
         7.2285]) 
 mean: tensor([9.9636e+01, 3.6870e+01, 6.2343e+01, 3.6324e+01, 1.6689e-01, 2.7655e+00,
        2.3217e+00, 7.2206e-02, 2.0575e+00])
G_solv
 std: tensor([2.8032], dtype=torch.float64) 
 mean: tensor([-5.1549], dtype=torch.float64)

length check-> Solvent: 2529, Solute: 2529, G_solv: 2529

Solvent
 std: tensor([ 0.0656,  0.2982,  0.2228, 24.6910, 28.1419,  0.3204,  0.2254]) 
 mean: tensor([ 1.4262,  0.1996,  0.2314, 48.5389, 17.9622,  0.1575,  0.0863])
Solute
 std: tensor([46.2081, 23.2195, 32.6769, 33.9146,  2.0706,  6.2217,  6.3596,  1.3146,
         7.2285]) 
 mean: tensor([9.9636e+01, 3.6870e+01, 6.2343e+01, 3.6324e+01, 1.6689e-01, 2.7655e+00,
        2.

In [4]:
print(dataset.norm_params)
try:
    os.makedirs(project_path('Solvation_1/Runs/' + runs_folder))
except FileExistsError:
    pass
with open(project_path('Solvation_1/Runs/' + runs_folder + '/norm_params.pkl'), 'wb+') as f:
    pkl.dump(dataset.norm_params, f)

{'Solvent': (tensor([ 0.0656,  0.2982,  0.2228, 24.6910, 28.1419,  0.3204,  0.2254]), tensor([ 1.4262,  0.1996,  0.2314, 48.5389, 17.9622,  0.1575,  0.0863])), 'Solute': (tensor([46.2081, 23.2195, 32.6769, 33.9146,  2.0706,  6.2217,  6.3596,  1.3146,
         7.2285]), tensor([9.9636e+01, 3.6870e+01, 6.2343e+01, 3.6324e+01, 1.6689e-01, 2.7655e+00,
        2.3217e+00, 7.2206e-02, 2.0575e+00])), 'G': (tensor([2.8032], dtype=torch.float64), tensor([-5.1549], dtype=torch.float64))}


## Train Network

In [5]:
x, y = next(iter(dataset))
x.shape

torch.Size([1, 16])

In [7]:
in_feat = next(iter(dataset))[0].shape[-1]
verbose = False
model = LinearNet3(in_features = in_feat)
loss_function = nn.MSELoss(reduction='sum')
optimizer = torch.optim.Adam(model.parameters())
model.train()

MSE = train(model,train_loader, val_loader, solvent_test_loader, solute_test_loader, loss_function, optimizer, epochs=10000, ckp_path=runs_folder)

KeyboardInterrupt: 

In [ ]:
plot_losses(project_path('Solvation_1/Runs/'+runs_folder+'/run_log.tsv'))